In [1]:
from google.colab import drive

drive.mount('/content/drive')

%cd /content/drive/MyDrive/ift6289-project/
!ls

Mounted at /content/drive
/content/drive/MyDrive/ift6289-project
'Copy of ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'Copy of ift6289-project-mt5-base-ENtoBN.ipynb'
 gemma_7b_finetune_template.ipynb
 hasan-etal-2020-low
 hasan-etal-2020-low.tar.bz2
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-BNtoEN.ipynb'
'ift6289-project-facebook mbart-large-50-many-to-many-mmt-ENtoBN.ipynb'
'ift6289-project-facebook nllb-200-distilled-600M.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-bn-en.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-en-mul.ipynb'
'ift6289-project-Helsinki-NLP opus-mt-mul-en.ipynb'
'ift6289-project-meta-llama Llama-2-7b-hf.ipynb'
 ift6289-project-mt5-base-BNtoEN.ipynb
 ift6289-project-mt5-base-ENtoBN.ipynb
 ift6289-project-umt5-base-BNtoEN.ipynb
 ift6289-project-umt5-base-ENtoBN.ipynb
 logs
 results


In [2]:
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install transformers
!pip install ml_things
!pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 9.2 MB/s eta 0:00:00
  Created wheel for ml_things: filename=ml_things-0.0.1-py3-none-any.whl size=24393 sha256=4071bc36b64a126e2330bb

In [3]:
from transformers import set_seed, AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, pipeline, MT5ForConditionalGeneration, T5Tokenizer, DataCollatorForSeq2Seq
from datasets import Dataset, load_dataset, load_metric
import torch
import pandas as pd
from tqdm.auto import tqdm  # tqdm is great for progress bars
import sacrebleu
import numpy as np

In [4]:
set_seed(42)

dataset = load_dataset("csebuetnlp/BanglaNMT")
# print(dataset)
# print(dataset['train'][0])
# print(dataset['train'][1])
# print(dataset['train'][2])
# print(dataset['train'][0]['bn'])
# print(dataset['train'][0]['en'])

training_subset_size = 100000
dataset["train"] = dataset["train"].select(range(training_subset_size))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/2379749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

In [5]:
print(dataset)
print(dataset['train'][0])
print(dataset['train'][1])
print(dataset['train'][2])
print(dataset['train'][0]['bn'])
print(dataset['train'][0]['en'])

DatasetDict({
    train: Dataset({
        features: ['bn', 'en'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en'],
        num_rows: 597
    })
})
{'bn': 'আমাকে সব কিছু জানাও', 'en': 'Just keep me informed.'}
{'bn': '"এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...', 'en': 'It\'s bad, "Amor and Psyche" is, master.'}
{'bn': 'কাছানো', 'en': 'Overtake'}
আমাকে সব কিছু জানাও
Just keep me informed.


In [6]:
# Specify the model checkpoint
# model_checkpoint = "facebook/nllb-200-distilled-600M"
# model_checkpoint = "Helsinki-NLP/opus-mt-bn-en"
model_checkpoint = "google/mt5-base"
tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [7]:
def preprocess_function(examples):
    inputs = ["Translate English to Bengali: " + ex for ex in examples["en"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    # Prepare the labels (Bengali translations)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['bn'], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)
print(tokenized_datasets)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/597 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['bn', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 597
    })
})


In [8]:
for i in range(3):  # Adjust the range as needed
    print(f"\nExample {i+1}:")
    print("Input (English):", tokenizer.decode(tokenized_datasets['train'][i]['input_ids'], skip_special_tokens=True))
    print("Label (Bengali):", tokenizer.decode(tokenized_datasets['train'][i]['labels'], skip_special_tokens=True))


Example 1:
Input (English): Translate English to Bengali: Just keep me informed.
Label (Bengali): আমাকে সব কিছু জানাও

Example 2:
Input (English): Translate English to Bengali: It's bad, "Amor and Psyche" is, master.
Label (Bengali): "এ্যামোর এন্ড সাইকি", এটা খারাপ, মনিব এটায় প্রচুর পরিমাণে চিরহরিৎ (একপ্রকার সুগন্ধী লতা) আছে আর প্রচুর পরিমাণে...

Example 3:
Input (English): Translate English to Bengali: Overtake
Label (Bengali): কাছানো


In [9]:
model = MT5ForConditionalGeneration.from_pretrained(model_checkpoint)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
# Load both BLEU and SacreBLEU metrics
bleu_metric = load_metric('bleu')
sacrebleu_metric = load_metric('sacrebleu')

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # # Compute BLEU score
    # bleu_result = bleu_metric.compute(predictions=decoded_preds, references=[decoded_labels])
    # Compute SacreBLEU score
    sacrebleu_result = sacrebleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Prepare result dictionary
    results = {
        # "bleu": bleu_result['score'],
        "sacrebleu": sacrebleu_result['score']
    }

    return results

<ipython-input-10-415e82ee0514>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for sacrebleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/sacrebleu/sacrebleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [11]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="sacrebleu",
    # report_to="none",
)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# # Pre-training test results
baseline_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Baseline Test Metrics:", baseline_test_results.metrics)
baseline_validation_results = trainer.evaluate()
print("Baseline Validation Metrics:", baseline_validation_results)

# Fine-tune the model
print("\nFine-tuning the model...")
trainer.train()
print("Model fine-tuning complete.")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Baseline Test Metrics: {'test_loss': 39.931156158447266, 'test_sacrebleu': 0.009967681438212821, 'test_runtime': 33.2843, 'test_samples_per_second': 30.044, 'test_steps_per_second': 1.893}


Baseline Validation Metrics: {'eval_loss': 39.695987701416016, 'eval_sacrebleu': 0.013624154009485724, 'eval_runtime': 19.6615, 'eval_samples_per_second': 30.364, 'eval_steps_per_second': 1.933}

Fine-tuning the model...


Epoch,Training Loss,Validation Loss,Sacrebleu
1,0.757400,0.424400,3.958439
2,0.450000,0.370976,5.974558
3,0.399300,0.353905,6.032228


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Model fine-tuning complete.


In [12]:
# Evaluate the model
validation_results = trainer.evaluate()
print("Final validation Metrics", validation_results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Final validation Metrics {'eval_loss': 0.35390469431877136, 'eval_sacrebleu': 6.0322279296181724, 'eval_runtime': 30.3024, 'eval_samples_per_second': 19.701, 'eval_steps_per_second': 1.254, 'epoch': 3.0}


In [13]:
# Calculate scores on the test set
final_test_results = trainer.predict(tokenized_datasets["test"], metric_key_prefix="test")
print("Final Test Metrics:", final_test_results.metrics)

Final Test Metrics: {'test_loss': 0.3490679860115051, 'test_sacrebleu': 6.287674806347149, 'test_runtime': 48.1426, 'test_samples_per_second': 20.772, 'test_steps_per_second': 1.309}


In [14]:
# Decode the predictions and labels
decoded_predictions = tokenizer.batch_decode(final_test_results.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(final_test_results.label_ids, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions[i]}")
    print(f"Label {i+1}: {decoded_labels[i]}")

Sample Predictions and Labels:

Prediction 1: যুক্তরাজ্যের বাংলাদেশ হাই কমিশনার সৈয়দা মুনা তাসনি
Label 1: বিমানবন্দরে যুক্তরাজ্যে নিযুক্ত বাংলাদেশ হাইকমিশনার সাঈদা মুনা তাসনীম ও লন্ডনে বাংলাদেশ মিশনের জ্যেষ্ঠ কর্মকর্তারা তাকে বিদায় জানান।

Prediction 2: সন্ত্রাসীদের কোন জাতি, ধর্ম এবং দেশ নেই।
Label 2: সন্ত্রাসীদের কোনো বর্ণ, ধর্ম ও দেশ নেই।

Prediction 3: ডারেক্ট ক্যাবেরা, কলকাতা বিশ্ববিদ্যালয়ের অধ্যাপক, একটি প্র
Label 3: কর্নেল বিশ্ববিদ্যালয়ের অধ্যাপক ডেরেক কাবেরা "শ্রেণিবদ্ধ চিন্তার অন্ধকার দিক" নামের একটি প্রবন্ধ রচনা করেছেন।

Prediction 4: তিনি বলেন যে, মানুষ কেজিতে ৪৫ টাকায় আলু,
Label 4: জনসাধারণ প্রতি কেজি পেঁয়াজ ৪৫ টাকায়, চিনি ৫০ টাকায়, মশুর ডাল ৫০ টাকায় ও প্রতি লিটার সয়াবিন তেল ৮৫ টাকায় কিনতে পারবে বলে জানান তিনি।

Prediction 5: শ্রীপুরের দুই ইউনিটের স্থানীয় ও ধ্বংসকারীরা
Label 5: শ্রীপুর ফায়ার স্টেশনের দুই ইউনিটের কর্মীরা প্রায় দুঘণ্টা চেষ্টা করে আগুন নেভায়।

Prediction 6: প্রধানমন্ত্রী শেখ হাসিনা রোমে রোমে আবু ঢিবিতে আসেন।
Label 6: প্রধানমন্ত্রী শেখ হাসিনা ইতা

# Test on SUPara-benchmark

In [15]:
supara_target_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_bn.txt"
supara_source_test_path = f"hasan-etal-2020-low/data/SUPara-benchmark/suparatest2018/suparatest2018_en.txt"

with open(supara_source_test_path, "r", encoding="utf-8") as f:
    supara_source_data = f.readlines()

with open(supara_target_test_path, "r", encoding="utf-8") as f:
    supara_target_data = f.readlines()

print(supara_source_data[0])
print(supara_target_data[0])

She likes red roses.

সে লাল গোলাপ পছন্দ করে।



In [16]:
supara_dataset = Dataset.from_dict({"en": supara_source_data, "bn": supara_target_data})
tokenized_supara_dataset = supara_dataset.map(preprocess_function, batched=True)
print(tokenized_supara_dataset)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['en', 'bn', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3921: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [17]:
final_test_results_supara = trainer.predict(tokenized_supara_dataset, metric_key_prefix="test")
print("Final test metrics SUPara:", final_test_results_supara.metrics)
print("Final test results SUPara:", final_test_results_supara)

Final test metrics SUPara: {'test_loss': 0.452079713344574, 'test_sacrebleu': 4.353064066643635, 'test_runtime': 24.2432, 'test_samples_per_second': 20.624, 'test_steps_per_second': 1.32}
Final test results SUPara: PredictionOutput(predictions=array([[    0,   259,  9342, ..., 33850,  2141,  1152],
       [    0,  1244,  3459, ...,     0,     0,     0],
       [    0,  5998,  3148, ...,     0,     0,     0],
       ...,
       [    0,  1244,  9156, ...,   259,  6713, 91728],
       [    0,   259,  8909, ...,  1433, 12922,  4830],
       [    0, 10447,  9725, ..., 46068,  1807, 60977]]), label_ids=array([[  4230,    259,  71980, ...,      0,      0,      0],
       [   259,   1413,  15940, ...,      0,      0,      0],
       [  5998,   3148,  11977, ...,      0,      0,      0],
       ...,
       [  1244,   4790,  10968, ...,      0,      0,      0],
       [  3154, 171423,   4381, ...,      0,      0,      0],
       [  2996,  32258,   4151, ...,      0,      0,      0]]), metrics={'

In [18]:
# Decode the predicted sequences
decoded_predictions_supara = tokenizer.batch_decode(final_test_results_supara.predictions, skip_special_tokens=True)
labels_supara = np.where(final_test_results_supara.label_ids != -100, final_test_results_supara.label_ids, tokenizer.pad_token_id)
decoded_labels_supara = tokenizer.batch_decode(labels_supara, skip_special_tokens=True)

# Print a sample of the predictions and labels
print("Sample Predictions and Labels:")
for i in range(10):  # Adjust the range to print more or fewer examples
    print(f"\nPrediction {i+1}: {decoded_predictions_supara[i]}")
    print(f"Label {i+1}: {decoded_labels_supara[i]}")

Sample Predictions and Labels:

Prediction 1: তিনি লাল রঙের রঙের ফুল পছন্দ করেন
Label 1: সে লাল গোলাপ পছন্দ করে।

Prediction 2: এটা আমাদের দেহকে স্থির করে।
Label 2: ইহা আমাদের শরীরকে দৃঢ় করে।

Prediction 3: আমরা বাংলাদেশে বাস করি।
Label 3: আমরা বাংলাদেশে বাস করি।

Prediction 4: মানুষের জীবন খুবই ছোট।
Label 4: মানব জীবন অত্যন্ত সংক্ষিপ্ত।

Prediction 5: ট্রাফিক পুলিশের বড় দায়িত্ব রয়েছে।
Label 5: ট্রাফিক পুলিশের অনেক দায়িত্ব থাকে।

Prediction 6: ক্রিকেট আমার প্রিয় খেলা।
Label 6: ক্রিকেট আমার প্রিয় খেলা।

Prediction 7: গণ সচেতনতা সৃষ্টি করা উচিত।
Label 7: গণসচেতনতা সৃষ্টি করা উচিত।

Prediction 8: কিছু সমস্যা থাকতে পারে।
Label 8: সেখানে কিছু সমস্যা থাকতে পারে।

Prediction 9: তার নাম কী?
Label 9: তার নাম কি?

Prediction 10: আমাদের কলেজ খুবই প্রাচীন।
Label 10: আমাদের কলেজ বহু প্রাচীন।
